In [169]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

#  отримаємо дані з файлу
data = pd.read_csv('Housing.csv')
X = data[['area', 'bathrooms', 'bedrooms']].values
y = data['price'].values


## Нормалізація- денормалізація даних

In [170]:
def normalize_data(X):
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)
    X_normalized = (X - X_min) / (X_max - X_min)
    return X_normalized

def de_normalize_data(X_origin,X_normilized):
    X_min = X_origin.min(axis=0)
    X_max = X_origin.max(axis=0)
    X_de_normalized = X_min + X_normilized*(X_max - X_min)
    return X_de_normalized

## напишіть функцію гіпотези лінійної регресії у векторному вигляді;

In [171]:
def hypothesis(X,W):
    return np.dot(X, W) # перемножити вектор вагів на вектор ознак

## створіть функцію для обчислення функції втрат у векторному вигляді;


In [192]:
def compute_loss(X, y, W):

    m = len(y)  # кількість рядків з даними
    predictions = hypothesis(X, W)  # розраховуємо гіпотезу
    errors = predictions - y  # розрахунок втрат
    loss = (1 / (2 * m)) * np.dot(errors.T, errors)  # середньо-квадратична функцыя втрат
    return loss

## реалізуйте один крок градієнтного спуску;

In [191]:
def gradient_descent(X, y, w, alpha):
    m = len(y)  # кількість рядків даних
    predictions = hypothesis(X, w)  # рахуємо гіпотезу
    errors = predictions - y  # рахуємо помилку, як різницю
    gradient = (1 / m) * np.dot(X.T, errors)  # розраховуємо градієнт в векторному виразі
    w = w - alpha * gradient  # оновлюємо коефіцієнти
    return w

## знайдіть найкращі параметри 𝑤⃗ w для датасету використовуючи написані вами функції, прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;

In [240]:
X_normalized = normalize_data(X)
X_normalized = np.c_[np.ones(X_normalized.shape[0]), X_normalized]
y_normalized = normalize_data(y)

w_gradient = np.random.rand(X_normalized.shape[1]) # для першої ітерації - довільні значення від 0 до 1

# задаємо початкові параметри
alpha = 0.1  # Learning rate
num_iterations = 20000  # Number of iterations
mse_loss = 1
prev_mse_loss = 2
stopping_treshold = 0.0000000000000001 # вихід з циклу по кількості ітерацій, або по покращенню функції втрат менше ніж на ...
i=1

while i<=num_iterations and (prev_mse_loss-mse_loss) > stopping_treshold:
    prev_mse_loss = mse_loss
    w_gradient = gradient_descent(X_normalized, y_normalized, w_gradient, alpha)
    mse_loss = compute_loss(X_normalized, y_normalized, w_gradient)
    # print(f'step: {i}, mse_loss {mse_loss}, prev_mse_loss {prev_mse_loss}, w: {w}') #перевірка покроково
    i +=1
    
y_gradient = hypothesis(X_normalized, w_gradient)
y_gradient = de_normalize_data(y,y_gradient)
print("Optimal parameters (gradient solution):", w_gradient)
print(f'y:          {y[0:10]}')
print(f'y_gradient: {y_gradient[0:10].astype(int)}')

Optimal parameters (gradient solution): [0.04282675 0.47714285 0.36001168 0.17611438]
y:          [13300000 12250000 12250000 12215000 11410000 10850000 10150000 10150000  9870000  9800000]
y_gradient: [ 7036628 10392013  7591861  7066929  5650583  8046150  8862038 12155035  5908142  5997270]


## знайдіть ці ж параметри за допомогою аналітичного рішення;

In [189]:
def analytical_solution(X, y):
    w = np.linalg.inv(X.T @ X) @ X.T @ y
    return w

X_normalized = normalize_data(X)
X_normalized = np.c_[np.ones(X_normalized.shape[0]), X_normalized]

y_normalized = normalize_data(y)

w_analytical = analytical_solution(X_normalized, y_normalized)
y_analytical = hypothesis(X_normalized,w_analytical)
y_analytical = de_normalize_data(y,y_analytical)
print("Optimal parameters (analytical solution):", w_analytical)
print(f'y:             {y[0:10]}')
print(f' analytical_y: {y_analytical[0:10].astype(int)}')



Optimal parameters (analytical solution): [0.0428274  0.47714269 0.36001286 0.17611257]
y:             [13300000 12250000 12250000 12215000 11410000 10850000 10150000 10150000  9870000  9800000]
 analytical_y: [ 7036627 10392020  7591864  7066928  5650577  8046157  8862041 12155033  5908136  5997273]


## для перевірки спрогнозованих значень, використайте LinearRegression з бібліотеки scikit-learn та порівняйте результати. 

In [188]:
np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:0.2f}'.format})


np.set_printoptions(edgeitems=3)
np.core.arrayprint._line_width = 80

model_sklearn = LinearRegression()

X_normalized = normalize_data(X)
y_normalized = normalize_data(y)

model_sklearn.fit(X_normalized, y_normalized)

coefficients_sklearn = model_sklearn.coef_
intercept_sklearn = model_sklearn.intercept_

y_sklearn = model_sklearn.predict(X_normalized)
y_sklearn = de_normalize_data(y,y_sklearn)


print(f"Optimal parameters (sklearn solution): {intercept_sklearn}, {coefficients_sklearn}")
print(f'        y: {y[0:10]}')
print(f'y_sklearn: {y_sklearn[0:10].astype(int)}')


Optimal parameters (sklearn solution): 0.04282739976995403, [0.47714269 0.36001286 0.17611257]
        y: [13300000 12250000 12250000 12215000 11410000 10850000 10150000 10150000  9870000  9800000]
y_sklearn: [ 7036627 10392020  7591864  7066928  5650577  8046157  8862041 12155033  5908136  5997273]


In [239]:
# all decision in 1 output:
print("Optimal parameters (gradient solution):  ", w_gradient)
print("Optimal parameters (analytical solution):", w_analytical)
print(f"Optimal parameters (sklearn solution):    {intercept_sklearn}, {coefficients_sklearn}")

Optimal parameters (gradient solution):   [0.04282807 0.4771423  0.36001408 0.17611083]
Optimal parameters (analytical solution): [0.0428274  0.47714269 0.36001286 0.17611257]
Optimal parameters (sklearn solution):    0.04282739976995403, [0.47714269 0.36001286 0.17611257]
